In [1]:
pip install --upgrade tensorflow

Requirement already up-to-date: tensorflow in c:\users\asus\anaconda3\lib\site-packages (2.5.0)
Note: you may need to restart the kernel to use updated packages.


In [2]:
#importing all the libraries
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from sklearn.model_selection import train_test_split
import time
from matplotlib import pyplot as plt
import os
import re

In [3]:
english_sentences= pd.read_json(r'C:\Users\Asus\Desktop\Training\trainEnglish.json')

In [4]:
english_sentences=english_sentences[['Heading']]
english_sentences.head()

,Heading
0,Andrea Masi opened the scoring in the fourth m...
1,Despite controlling the game for much of the f...
2,Portugal never gave up and David Penalva score...
3,Italy led 16-5 at half time but were matched b...
4,However Bortolussi scored his fourth penalty o...


In [5]:
print(english_sentences.shape)
english_sentences= english_sentences['Heading'].tolist()


(20105, 1)


In [6]:
hindi_sentences= pd.read_json(r'C:\Users\Asus\Desktop\Training\trainHindi.json') 
hindi_sentences= hindi_sentences[['Heading']]  
hindi_sentences.head()

,Heading
0,एंड्रिया मैसी ने चौथे मिनट में इटली के लिए ट्र...
1,पहले हाफ़ के अधिकांश समय तक खेल नियंत्रण में र...
2,"हार न मानते हुए, पुर्तगाल के डेविड पेनलावा ने ..."
3,पहले हाफ़ में इटली 16-5 से आगे था पर दूसरे हाफ...
4,हालाँकि बोर्तोलुसी ने मैच का चौथा पेनाल्टी स्क...


In [7]:
hindi_sentences.head()

,Heading
0,एंड्रिया मैसी ने चौथे मिनट में इटली के लिए ट्र...
1,पहले हाफ़ के अधिकांश समय तक खेल नियंत्रण में र...
2,"हार न मानते हुए, पुर्तगाल के डेविड पेनलावा ने ..."
3,पहले हाफ़ में इटली 16-5 से आगे था पर दूसरे हाफ...
4,हालाँकि बोर्तोलुसी ने मैच का चौथा पेनाल्टी स्क...


In [8]:
print(hindi_sentences.shape)
hindi_sentences= hindi_sentences['Heading'].tolist()


(20105, 1)


before going fprward and apply encoder and decoder, data needs to be preprocessed.
In this dataset there may be sentences with empty string, extra spaces, uppercase letters etc.

For preprocessing the data, we will first take care of the extra spaces, replace eveything with space except a-z, A-Z,'?','.','!',','.
we will add 'sentencestart' & 'sentenceend' at the front and back respectively, so that the model knows when does the sentence end and start.

In [9]:
def preprocess_sentence(sentence, english):
    if (type(sen) != str):
        return ''
    
    sentence = sentence.strip('.')   #remove the '.' in the sentence
    
    #The re.sub() function is used to replace occurrences of a particular sub-string with another sub-string. 
    #This function takes as input the following:
    #The sub-string to replace, The sub-string to replace with, The actual string
    
    sentence = re.sub(r"([?.!,¿;।])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)

    #now, Replacing everything except alphabets (upper & lower case both), ",", ".","?","!"."'"
    if(english == True):
        sentence = re.sub(r"[^a-zA-Z?.!,¿']+", " ", sentence)
        sentence = sentence.lower()      #changing all uppercases to lowercase

    sentence = sentence.strip()
    sentence = 'sentencestart ' + sentence + ' sentenceend'
 
    sentence = ' '.join(sentence.split())
    return sentence

In [10]:
#just checking if my function works
sen = "I am ? a /// gil .. who is NOT."
t = preprocess_sentence(sen,True)
print(t)

sentencestart i am ? a gil . . who is not sentenceend


In [12]:
#lets set maximum number of words in a sentence
num_words = 10000
MAX_WORDS_IN_A_SENTENCE = 100
english_vocab_size = num_words + 1
hindi_vocab_size = num_words + 1
test_ratio = 0.2
embedding_dim = 64
hidden_units = 1024
learning_rate = 0.006
epochs = 100

In [13]:
print(len(english_sentences))

20105


going  through all data points to make sure that no empty strings are considerend and no string has values more than MAX_WORDS_IN_A_SENTENCE

In [14]:
processed_e_sentences = []
processed_h_sentences = []

for i in range(len(english_sentences)):
    s = preprocess_sentence(english_sentences[i],True)
    processed_e_sentences.append(s)

    
for i in range(len(hindi_sentences)):
    s = preprocess_sentence(hindi_sentences[i],False)
    processed_h_sentences.append(s)

In [15]:
#exploring and checking what tweaks we made above

len(processed_h_sentences)

20105

now vetorize the text corpus.
fit_on_texts() assigns a unique index to each word. 
texts_to_Sequence() convert a text sentence to a list of numbers or a vector where the number points to the unique index of words.


we will use pad_sequences() to make sure that all of the vetors are of same length, by appending oov_token (out-of-vocab) just enough number of times

In [16]:
num_words = 10000
oov_token = '<UNK>'

In [17]:
def tokenize_sentences(processed, num_words, oov_token):
    tokenizer = Tokenizer(num_words = num_words, oov_token = oov_token)
    tokenizer.fit_on_texts(processed)
    word_index = tokenizer.word_index
    sequences = tokenizer.texts_to_sequences(processed)
    sequences = pad_sequences(sequences, padding = 'post')
    return word_index, sequences, tokenizer

In [18]:
i,s,t = tokenize_sentences(processed_e_sentences[0], num_words, oov_token)
print(i)


{'<UNK>': 1, 'e': 2, 't': 3, 'n': 4, 'a': 5, 'r': 6, 'i': 7, 's': 8, 'o': 9, 'h': 10, 'c': 11, 'd': 12, 'm': 13, 'f': 14, 'u': 15, 'y': 16, 'p': 17, 'g': 18, 'w': 19, 'l': 20}


In [19]:
print(processed_e_sentences[0])

sentencestart andrea masi opened the scoring in the fourth minute with a try for italy sentenceend


In [20]:
english_word_index, english_sequences, english_tokenizer = tokenize_sentences(processed_e_sentences, num_words, oov_token)
hindi_word_index, hindi_sequences, hindi_tokenizer = tokenize_sentences(processed_h_sentences, num_words, oov_token)

splitting the data set into training and validation sets

In [21]:
english_train_sequences, english_val_sequences, hindi_train_sequences, hindi_val_sequences = train_test_split(english_sequences, hindi_sequences, test_size = test_ratio)

In [22]:
BUFFER_SIZE = len(english_train_sequences)

In [23]:
size = len(english_train_sequences)
print(size)

16084


Divind the dataset into batches

Also we define Batch size:
The batch size is a number of samples processed before the model is updated. The number of epochs is the number of complete passes through the training dataset. The size of a batch must be more than or equal to one and less than or equal to the number of samples in the training dataset.


In [24]:
BATCH_SIZE = 512

In [25]:
#With the help of tf.data.Dataset.from_tensor_slices() method, we can get the slices of an array in the form of objects
dataset = tf.data.Dataset.from_tensor_slices((english_train_sequences, hindi_train_sequences)).shuffle(BATCH_SIZE)

In [26]:
dataset = dataset.batch(BATCH_SIZE, drop_remainder = True)

In [27]:
# as_numpy_iterator Returns an iterator which converts all elements of the dataset to numpy.

print(len(list(dataset.as_numpy_iterator())))

31


I used Seq2Seq architecture and have followed this tutorial https://www.tensorflow.org/addons/tutorials/networks_seq2seq_nmt

Now it involves 2 LSTM (Long short term memory), one for encoding and other for decoding. But, I would rather use GRU (Gated Recurrent Units) for both encoding and decoding since it takes less computation power, and we can achieve similar results.

# Encoding

1) each word in the input sentence is embedded and are represented in a different space embedding_dim (hyperparamter) dimensions. Hence, words like (girl, woman), (Book, Novel) are located closer in this space. So, the word "girl" has a chance of getting predicted same as chance of "woman", since they hold similar meanings

2) Now this embedded sentence is passed to GRU. The final hidden state of encoder GRU becomes the maiden state for decoder GRU. This final hidden state of encoder GRU, consists of encoding or information about the original/ source sentence. 

In [28]:
class Encoder(tf.keras.Model):
    def __init__(self, english_vocab_size, embedding_dim, hidden_units):
        super(Encoder, self).__init__()
        self.embedding = tf.keras.layers.Embedding(english_vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(hidden_units, return_sequences = True, return_state = True)
    
    def call(self, input_sequence):
        x = self.embedding(input_sequence)
        encoder_sequence_output, final_encoder_state = self.gru(x)
        return encoder_sequence_output, final_encoder_state

In [29]:
#Initialize encoder

encoder = Encoder(english_vocab_size, embedding_dim, hidden_units)

# Decoder (without Attention)

the target sentence is generated by the decoder GRU network language model. As discussed before, the final encoding hidden state is used as the initial hidden state for the decoder GRU. 
'sentencestart' is given to the GRU as the first input, to predict the next. This token is used to predict probability of occuerance of all num_words (10000) number of words. The word with maximum probabilty becomes the next input of GRU, and it is all repeated will we reach sentenceend

Problems with this approach:

Since the encoder's final state becomes the initial hidden state for the decoder,so all the information of the information of the source has to be compressed in the final state. It might be biased to the information at the end of the sentence. And this is an issue!

So to solve this, encoder final hidden state is not just taken into consideration, WEIGHTED SUM of all the outputs from the encoder is used. 

to solve it, I used ATTENTION

# Attention

Attention literaaly gives weight to each word in a sentence. When there is a need to know which encoder output holds the similar informationn as in the decoder hidden state Attention is used. 

Vector of Attention Scores are calculated at each step when a word is being predicted at the GRU cell in the decoder. This vector determines the weightage of each encoder output to find the weighted sum

By letting the decoder have an attention mechanism, we relieve the encoder from the burden of having to encode all information in the source sentence into a fixed-length vector.

I have used the "Basic Dot Product Attention". The name is quite self-explanatory, it is the dot product of the input matrices.

The basic dot product attention has an assumption. According to it, the dimensions of both the input matrix on the axis where the dot product is to be taken, needs to be same. This dimention in my implemnetation is given by hidden_units, which os same for encoder and decoder

In [30]:
class BasicDotProductAttention(tf.keras.layers.Layer):
    def __init__(self):
        super(BasicDotProductAttention, self).__init__()

    def call(self, decoder_hidden_state, encoder_outputs):
       #Dimensions of decoder_hidden_state => (BATCH_SIZE, hidden_units)
       #Dimensions of encoder_outputs => (BATCH_SIZE, MAX_WORDS_IN_A_SENTENCE, hidden_units)

        decoder_hidden_state_with_time_axis = tf.expand_dims(decoder_hidden_state, 2)
        #Dimensions of decoder_hidden_state_with_time_axis => (BATCH_SIZE, hidden_units, 1)
        attention_scores = tf.matmul(encoder_outputs, decoder_hidden_state_with_time_axis)
        #Dimensions of attention_scores => (BATCH_SIZE, MAX_WORDS_IN_A_SENTENCE, 1)
        attention_scores = tf.nn.softmax(attention_scores, axis = 1)
        weighted_sum_of_encoder_outputs = tf.reduce_sum(encoder_outputs * attention_scores, axis = 1)
        #Dimensions of weighted_sum_of_encoder_outputs => (BATCH_SIZE, hidden_units)

        return weighted_sum_of_encoder_outputs, attention_scores
        

# Decoder (with attention)

in Decoder class:

like in encoder, here is also an embedding layer for sequences in the to-be-converted-into language, here it is Hindi. each word in a sequence is represented in the embedding space with similar meaning words are placed closer

after calling the attnetion layer, weighted sum of encoder outputs is calculated

the result (representation of embedding space and weighted sum of encoder output), is abtained and is sent to the GRU layer of the decoder

the output of the GRU layer is then sent to the dense layer which gives the probability of  occurrence of all of the hindi_vocab_size number of words. Word with high probability implies that the model thinks that this word should be the next word.

In [31]:
class Decoder(tf.keras.Model):
    def __init__(self, hindi_vocab_size, embedding_dim, hidden_units):
        super(Decoder, self).__init__()
        self.embedding = tf.keras.layers.Embedding(hindi_vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(hidden_units, return_state = True)
        self.word_probability_layer = tf.keras.layers.Dense(hindi_vocab_size, activation = 'softmax')
        self.attention_layer = BasicDotProductAttention()
        
    def call(self, decoder_input, decoder_hidden, encoder_sequence_output):
        x = self.embedding(decoder_input)
        
        
        weighted_sum_of_encoder_outputs, attention_scores = self.attention_layer(decoder_hidden, encoder_sequence_output)
        
        
        x = tf.concat([weighted_sum_of_encoder_outputs, x], axis = -1)
        x = tf.expand_dims(x, 1)
        
        
        decoder_output, decoder_state = self.gru(x)
        
        
        word_probability = self.word_probability_layer(decoder_output)
        
        
        return word_probability, decoder_state, attention_scores

In [32]:
#initialize the decoder
decoder = Decoder(hindi_vocab_size, embedding_dim, hidden_units)

# process till now....

1) find the encoder output and encoder hidden state from the encoder oject. these outputs is used to find the attention score. the encoder final hidden state is the initial hidden state of decoder

2) to predict the next word in hindi, an input word, previous decoder hidden state, encoder sequence outputs are given to the decoder object. word prediction probabilty and current hidden state are returned

3) maximum probabilty word is considered as the input for the next decoder GRU cell. the current decoder hidden state becomes the input hudden state for the next decoder GRU cell.

4) loss is calculated using the word prediction probability and actual word in the target sentence

in every epoch, these steps are called for all the batches & loss for all the epochs is stored

sparse categorical crossentropy loss and adam optimizer are chosen!

In [33]:
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(reduction='none')
def loss_function(actual_words, predicted_words_probability):
    loss = loss_object(actual_words, predicted_words_probability)
    mask = tf.where(actual_words > 0, 1.0, 0.0)
    return tf.reduce_mean(mask * loss)


def train_step(english_sequences, hindi_sequences):
    loss = 0
    with tf.GradientTape() as tape:
        encoder_sequence_output, encoder_hidden = encoder(english_sequences)
        decoder_hidden = encoder_hidden
        decoder_input = hindi_sequences[:, 0]
        for i in range(1, hindi_sequences.shape[1]):
            predicted_words_probability, decoder_hidden, _ = decoder(decoder_input, decoder_hidden, encoder_sequence_output)
            actual_words = hindi_sequences[:, i]
            
            if np.count_nonzero(actual_words) == 0:
                break
            loss += loss_function(actual_words, predicted_words_probability)

            decoder_input = actual_words

    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    return loss.numpy()
all_epoch_losses = []
training_start_time = time.time()



In [34]:
for epoch in range(epochs):
    epoch_loss = []
    start_time = time.time()
    for(batch, (english_sequences, hindi_sequences)) in enumerate(dataset):
        batch_loss = train_step(english_sequences, hindi_sequences)
        epoch_loss.append(batch_loss)
    
    all_epoch_losses.append(sum(epoch_loss)/len(epoch_loss))
    print("Epoch No.: " + str(epoch) + " Time: " + str(time.time()-start_time))
    
print("All Epoch Losses: " + str(all_epoch_losses))
print("Total time in training: " + str(time.time() - training_start_time))

plt.plot(all_epoch_losses)
plt.xlabel("Epochs")
plt.ylabel("Epoch Loss")
plt.show()

Epoch No.: 0 Time: 1273.6945250034332
Epoch No.: 1 Time: 1613.1667745113373
Epoch No.: 2 Time: 1539.1818432807922
Epoch No.: 3 Time: 957.1024177074432
Epoch No.: 4 Time: 955.8483700752258
Epoch No.: 5 Time: 950.5274212360382
Epoch No.: 6 Time: 952.3498854637146
Epoch No.: 7 Time: 955.741516828537
Epoch No.: 8 Time: 952.1819429397583
Epoch No.: 9 Time: 951.3708064556122
Epoch No.: 10 Time: 954.2991886138916
Epoch No.: 11 Time: 951.8446118831635
Epoch No.: 12 Time: 951.538083076477
Epoch No.: 13 Time: 952.9957630634308
Epoch No.: 14 Time: 949.7646601200104
Epoch No.: 15 Time: 951.6074774265289
Epoch No.: 16 Time: 946.1992561817169
Epoch No.: 17 Time: 952.9822399616241
Epoch No.: 18 Time: 950.4610857963562
Epoch No.: 19 Time: 4120.194446802139
Epoch No.: 20 Time: 1456.2308254241943
Epoch No.: 21 Time: 1368.2881054878235
Epoch No.: 22 Time: 1749.5814476013184
Epoch No.: 23 Time: 12891.44866681099
Epoch No.: 24 Time: 1302.8859837055206
Epoch No.: 25 Time: 1091.2967038154602
Epoch No.: 26 Ti

KeyboardInterrupt: 

I trained for 12 hours+, but still the training was not complete, so I interupter the kernal, so that I can move forward with the testing

# testing the data

1) preprocess the input i.e the sentence which is to be translated

2) so now the above ouput is fed to trained encoder which return the encoded sequence output, and encoder's final hidden state.

3) eencoder's final state is the decoder's first hidden state and the first input to the decoder is a start token "sentencestart"

4) decoder return the predicted word probabilities. Maximum probability word is appened to the required final hindi sentence.

5) it goes on till we reach "sentenceend".

In [37]:
def get_sentence_from_sequences(sequences, tokenizer):
    return tokenizer.sequences_to_texts(sequences)

# Testing start!

def translate_sentence(sentence):
    sentence = preprocess_sentence(sentence, True)
    sequence = english_tokenizer.texts_to_sequences([sentence])[0]
    sequence = tf.keras.preprocessing.sequence.pad_sequences([sequence], maxlen=MAX_WORDS_IN_A_SENTENCE, padding='post')
    encoder_input = tf.convert_to_tensor(sequence)
    encoder_sequence_output, encoder_hidden = encoder(encoder_input)
    decoder_input = tf.convert_to_tensor([hindi_word_index['sentencestart']])
    decoder_hidden = encoder_hidden
    
    sentence_end_word_id = hindi_word_index['sentenceend']
    hindi_sequence = []
    for i in range(MAX_WORDS_IN_A_SENTENCE*2):
        predicted_words_probability, decoder_hidden, _ = decoder(decoder_input, decoder_hidden, encoder_sequence_output)
        #taking the word with maximum probability
        predicted_word_id = tf.argmax(predicted_words_probability[0]).numpy()
        hindi_sequence.append(predicted_word_id)
        #if the word 'sentenceend' is predicted, exit the loop
        if predicted_word_id == sentence_end_word_id:
            break
        decoder_input = tf.convert_to_tensor([predicted_word_id])
    print(sentence)
    return get_sentence_from_sequences([hindi_sequence], hindi_tokenizer)


    

In [42]:
print(translate_sentence("They said that this is not possible"))

sentencestart they said that this is not possible sentenceend
['उन्होंने कहा कि यह नहीं है । sentenceend']
